In [1]:
from nltk.corpus import gutenberg
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
text = macbeth.replace('\n', '')
import sys
from collections import defaultdict

class Importance_calculations:

    def __init__(self, file):
        self.data = self.filter(file)

    def filter(self, str):
        str = str.lower()
        i = [x for x in range(97)] 
        j = [x for x in range(123,256)]
        k = i + j
        for c in [chr(i) for i in k]:
            str = str.replace(c, '')
        return str

    def importance_dictionary(self):
        dictionary = []
        counter = defaultdict(int)
        phrase = ""
        for char in self.data:
            new_phrase = phrase + char
            
            if new_phrase in dictionary:
                phrase = new_phrase
            else:
                counter[phrase] += 1
                dictionary.append(new_phrase)
                # phrase = char
                phrase = ''
        print('importance dictionary:', dictionary)
        return counter, dictionary

    def rank(self):
        counter, dictionary = self.importance_dictionary()
        dictionary.sort(key=lambda str: len(str)*counter[str], reverse=True)
        return dictionary, counter
    
    def importance_sorting(self):
        ranked_dict, counter = self.rank()
        print(counter)
        return ranked_dict, counter
    
    def encode_rank(self):
        ranked_dict, _ = self.rank()
        ranking_levels = {}
        rank = 1
        for e in ranked_dict:
            ranking_levels[e] = rank
            rank += 1
        print('ranking_levels:', ranking_levels)
        return ranking_levels


#file = open('no_rel/data.txt', 'r')
#text = file.readlines()[0]
text = "aababbabbaababba"
#text = "aababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababbaaababbabbaababba"
Test = Importance_calculations(text)
rectified = Test.filter(text)
z,occ = Test.importance_sorting()
# print(z)
# print(occ)


u = Test.encode_rank()
print(sys.getsizeof(rectified))

importance dictionary: ['a', 'ab', 'abb', 'abba', 'aba', 'b', 'ba']
defaultdict(<class 'int'>, {'': 2, 'a': 1, 'ab': 2, 'abb': 1, 'b': 1, 'abba': 0, 'aba': 0, 'ba': 0})
importance dictionary: ['a', 'ab', 'abb', 'abba', 'aba', 'b', 'ba']
ranking_levels: {'ab': 1, 'abb': 2, 'a': 3, 'b': 4, 'abba': 5, 'aba': 6, 'ba': 7}
65


In [2]:
#encode_order = {'ab': 1, 'abb': 2, 'a': 3, 'b': 4, 'abba': 5, 'aba': 6, 'ba': 7}
encode_order = u
#text = "aababbabbaababba"
#text = rectified

class Encoding:

    def __init__(self, phrase, rankings_encode):
        self.data = phrase 
        self.encode_order = rankings_encode
        print(rankings_encode)

    def encoder(self):

        accum = ''
        encoded = ''
        appeared_before = []
        code_with_space = []
        for s in self.data:
            #print(accum)
            pattern = accum+s
            if pattern in appeared_before:
                accum = pattern 
            else:
                appeared_before.append(pattern)
                # print('appeared', appeared_before)
                if len(pattern) == 1:
                    encoded += '# '+pattern+' '
                    code_with_space.append("#"+pattern) 
                else:
                    encoded = encoded + str(self.encode_order[accum]) +' '+s+' '
                    code_with_space.append(str(self.encode_order[accum])+s)
                accum = ''
        #print(address_code)
        return encoded, appeared_before

    def encoded(self):
        code,_ = self.encoder()
        return code.replace(' ', '')

    def ambiguity_solver(self,):
        code_with_space,list_code = self.encoder()
        accumulator = code_with_space.split()
        running_dictionary = {}
        unambiguous = ''
        #print(accumulator)

        def return_key(dictionary, idx):
            return list(dictionary.keys())[list(dictionary.values()).index(idx)]

        for i in range(len(list_code)-1):
            s = accumulator[2*i], accumulator[2*i+1] 
            if s[0] != '#' :
                if (accumulator[2*i+2] != '#') and (int(accumulator[2*i+2]) not in list(running_dictionary.values())):
                    unambiguous += ''.join(s)
                    prev = return_key(running_dictionary, int(s[0]))
                    running_dictionary[prev + s[1]] = int(accumulator[2*i+2])
                elif (accumulator[2*i+2] != '#') and (int(accumulator[2*i+2]) in list(running_dictionary.values())):
                    prev = return_key(running_dictionary, int(s[0]))
                    actual_rank = self.encode_order[prev+s[1]]
                    running_dictionary[prev + s[1]] = actual_rank
                    unambiguous = unambiguous+''.join(s)+'('+str(actual_rank)+')'      
                else:
                    prev = return_key(running_dictionary, int(s[0]))
                    actual_rank = self.encode_order[prev + s[1]]
                    running_dictionary[prev + s[1]] = actual_rank
                    unambiguous = unambiguous + ''.join(s) + '(' + str(actual_rank) + ')'
            if s[0] == '#':
                if accumulator[2*i+2] == '#':
                    actual_rank = self.encode_order[s[1]]
                    running_dictionary[s[1]] = actual_rank
                    unambiguous = unambiguous + ''.join(s) + '(' + str(actual_rank) + ')'
                elif accumulator[2*i+2] != '#' and (int(accumulator[2*i+2]) in list(running_dictionary.values())) :
                    actual_rank = self.encode_order[s[1]]
                    running_dictionary[s[1]] = actual_rank
                    unambiguous = unambiguous+''.join(s)+'('+str(actual_rank)+')'
                else:
                    actual_rank = int(accumulator[2*i+2])
                    running_dictionary[s[1]] = actual_rank 
                    unambiguous = unambiguous + ''.join(s)
        unambiguous += accumulator[-2]+accumulator[-1]
        #print(running_dictionary)
        return unambiguous

#test = Encoding(text, encode_order)
test = Encoding(rectified, encode_order)
code = test.encoded()
clear_code = test.ambiguity_solver()

print(sys.getsizeof(code))
print(code)
print(test.ambiguity_solver())
print(sys.getsizeof(test.ambiguity_solver()))
#test.ambiguities()

{'ab': 1, 'abb': 2, 'a': 3, 'b': 4, 'abba': 5, 'aba': 6, 'ba': 7}
63
#a3b1b2a1a#b4a
#a3b1b2a(5)1a(6)#b4a
69


In [3]:
class Decoding:

    def __init__(self, unambiguous):
        self.code = unambiguous 
    
    def nearest_closing_bracket(self, idx):
        j = 1
        while self.code[idx+j] != ')':
            j+=1
        return idx+j, self.code[idx+1:idx+j]
    
    def farthest_ending_digit(self, idx):
        j = 0
        while self.code[idx+j].isnumeric():
            j += 1
        return idx+j-1, self.code[idx:idx+j]
    
    def return_key(self, dictionary, idx):
        return list(dictionary.keys())[list(dictionary.values()).index(idx)]
    
    def decoder(self):
        dictionary = {}
        decode = ''
        i = 0
        while i < len(self.code):
            if self.code[i] == '#':
                if self.code[i+2] == '(': 
                    ending_bracket, inside = self.nearest_closing_bracket(i+2)
                    dictionary[self.code[i+1]] = int(inside)
                    i = 1+ending_bracket
                    decode += self.code[i+1]
                else:
                    ending_location, address = self.farthest_ending_digit(i+2)
                    dictionary[self.code[i+1]] = int(address)
                    i = 1 + ending_location
                    decode += self.code[i+1]
            else:
                j, prev = self.farthest_ending_digit(i)
                if self.code[j+2] == '(':
                    ending_bracket, inside = self.nearest_closing_bracket(j+2)
                    dictionary[self.return_key(dictionary, int(prev))+self.code[j+1]] = int(inside)
                    decode += self.return_key(dictionary, int(prev))+self.code[j+1]
                    i = 1+ending_bracket
                else:
                    ending_location, address = self.farthest_ending_digit(j+2)
                    dictionary[self.return_key(dictionary, int(prev))+self.code[j+1]] = int(address)
                    decode += self.return_key(dictionary, int(prev))+self.code[j+1]
                    i = 1 + ending_location
        print(dictionary)
        return decode

decoder = Decoding(clear_code)
decoded = decoder.decoder()
print(decoded)


ValueError: invalid literal for int() with base 10: ''